# RP 55

In [1]:
from bayesian_models import Pbnn
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from doepy import build
import pickle
from scipy.stats import norm

In [2]:
# https://rprepo.readthedocs.io/en/latest/_modules/gfun_55.html#gfun_55
# RP55
# Table 21 – Challenge set 2.
# set_id - problem_id
# 2 - 4

import gfun_55_mod as gfun_55_mod

function = gfun_55_mod.gfun_55_mod  #assigning ground truth sim

Active Train

In [3]:
number_experiments = 10
passive_samples = 10 
active_samples = 50

dim = 2
output = 1
layers, archit = 2 , [16,16]  #change archit size if layers are increased
bnn_simulations = 500
training_epochs = 15000
n_mcs = int(1e6)

for experiments in range(number_experiments):
    print('Experiment: ', experiments+1 , '########################################################' )
    
    ActiveTrain_1 = {}

    #PASSIVE TRAINING---------------------------------------------------
    X = np.zeros((passive_samples, dim))
    #-------------------------------------------------------------------LHS
    Xdoe = build.space_filling_lhs({ 'x1':[-1, 1],      
                                     'x2':[-1, 1],},
                                     num_samples = passive_samples)
    X[:,0] = Xdoe['x1']
    X[:,1] = Xdoe['x2']

    Y = function(X)

    #ACTIVE TRAIN LOOP ---------------------------------------------------
    for active in range(passive_samples, active_samples+passive_samples+1):

        n_doe = len(X)
        batch_size = [np.floor_divide(n_doe, 2)][0] + 1  # half of the DoE
        
        #-------------------------------------------creating bnn
        #setting up the network architecture -----------------------------------
        config = {"n_infeatures": dim,
              "n_outfeatures": output,
              "n_samples": n_doe,
              "learn_all_params": False,  #to learn mean and sigma
              "fixed_param": 0.001} 
        
        ModelName = 'BNN_' + str(n_doe)
        mybnn = Pbnn(config)
        
        mybnn.build_bnn(layers, archit) #----------------------------------------------------------MODEL ARCHITECTURE
        #-------------------------------------------training bnn
        train_env = {"batch_size": batch_size,
                 "learning_rate": 0.001,
                 "epochs": training_epochs,
                 "callback_patience": 4000,
                 "verbose": 0}
        print(' ')
        print('Training BNN With', active, 'samples' )
        mybnn.train_bnn(X, Y, train_env)
        #-------------------------------------------MC population
        Xtest = np.random.uniform(-1.0, 1.0, size=(n_mcs, dim))

        #-------------------------------------------model predictions over MC population
        print('BNN predictions with MC population...')
        Mean_muY_MC, Stdv_muY_MC, Mean_sigmaY_MC, Stdv_sigmaY_MC = mybnn.modeluq_bnn(Xtest, nsim = bnn_simulations)

        PF = np.sum(Mean_muY_MC < 0) / n_mcs
        B = - norm.ppf( PF )

        print('PF =', PF, 'and B =',"%.5f" % round(B, 3) , 'with', active, 'training samples--------------------------------')
        #-------------------------------------------Selecting new training point
        U_f = np.abs(Mean_muY_MC) / Stdv_muY_MC
        U_min = np.argmin(U_f)
        X_new = Xtest[U_min]
        X = np.concatenate((X, X_new.reshape(-1, dim)), axis=0)

        Y_new = function(X_new)
        Y = np.concatenate((Y, Y_new), axis=0)

        #-------------------------------------------Saving results
        ActiveTrain_1[ModelName] = mybnn.weights, PF, B
        filename1 = 'Batch_'+ str(experiments+1)+'.sav'
    
        pickle.dump(ActiveTrain_1, open(filename1, 'wb'))


Experiment:  1 ########################################################


c:\Users\Administrator\anaconda3\envs\deep_learning\lib\site-packages\tensorflow_probability\python\layers\util.py:99: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use the `layer.add_weight()` method instead.
  loc = add_variable_fn(
c:\Users\Administrator\anaconda3\envs\deep_learning\lib\site-packages\tensorflow_probability\python\layers\util.py:109: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use the `layer.add_weight()` method instead.
  untransformed_scale = add_variable_fn(


 
Training BNN With 10 samples
Training completed
Minimum loss:  38.14536666870117
BNN predictions with MC population...
PF = 0.0 and B = inf with 10 training samples--------------------------------
 
Training BNN With 11 samples
Training completed
Minimum loss:  28.890958786010742
BNN predictions with MC population...
PF = 0.006387 and B = 2.49000 with 11 training samples--------------------------------
 
Training BNN With 12 samples
